<a href="https://colab.research.google.com/github/hiranlowe/FitionModel/blob/main/Fition_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries




In [22]:

import os
from google.colab import drive

import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation, Bidirectional, Conv1D, MaxPool1D, BatchNormalization, Dense, LSTM

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

import zipfile


Initialize TPU

In [23]:
# Function to get hardware strategy
def get_hardware_strategy():
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.config.optimizer.set_jit(True)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    return tpu, strategy

tpu, strategy = get_hardware_strategy()


Running on TPU  grpc://10.69.168.146:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


CSI packet processing function

In [24]:
def _read_csi_next(pcapfile, csi_size):
    """
    Note: Designed for internal use only.
    
    Parameters
    ----------
        pcapfile : File Object
        csi_size : Expected length of CSI in bytes. NFFT * 4
    """

    # Read Frame Size
    pcapfile.seek(8, os.SEEK_CUR)
    frame_size = int.from_bytes(
        pcapfile.read(4),
        byteorder = 'little',
        signed = False
    )

    # Skip some stuff
    pcapfile.seek(56, os.SEEK_CUR)

    # Read CSI data
    pcapfile.seek(8, os.SEEK_CUR)
    csi = np.frombuffer(
        pcapfile.read(csi_size), 
        dtype = np.int16,
        count = int(csi_size / 2)
    )

    # Skip any zero-padding
    pcapfile.seek((frame_size - csi_size - 60), os.SEEK_CUR)

    return csi

def read_csi(pcap_file_path):
    """
    You have to remove 
    null and pilot subcarriers
    yourself.

    Parameters
    ----------
        pcap_file_path : str
    """

    bandwidth = 80

    NFFT = int(bandwidth * 3.2) # Number of channels in FFT
    chunksize = 1024

    csi  = np.zeros((chunksize, NFFT * 2), dtype = 'int16')

    with open(pcap_file_path, 'rb') as pcapfile:
        filesize = os.stat(pcap_file_path).st_size
        pcapfile.seek(24, os.SEEK_SET)

        npackets = 0
        while pcapfile.tell() < filesize:
            if not (npackets % chunksize):
                csi = np.vstack((csi, np.zeros((chunksize, NFFT * 2), dtype = 'int16')))

            csi[npackets] = _read_csi_next(pcapfile, NFFT * 4)
            
            npackets += 1

    # Convert CSI complex numbers to Magnitude.
    csi_converted = np.abs(
        np.fft.fftshift(csi[:npackets, ::2] + 1.j * csi[:npackets, 1::2], axes=(1,))
    )

    return csi_converted

Processing data set files


> Import files -> call the above function -> remove unrelevant subcarriers 



In [25]:
drive.mount("/content/gdrive")
with zipfile.ZipFile('/content/gdrive/My Drive/Fition Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('')

movement_data=[]
values={0:"nm",1:"sitdown", 2:"standup",3:"walking",4:"falling",5:"getintobed"}
for key, value in values.items():
    for i in range(1,31):
        for j in range(1,5):
            file_path = "Fition Dataset/"+value+"_user"+str(j)+"_rm"+str(j)+"/"+value+"_user"+str(j)+"_rm"+str(j)+"_"+str(i)+".pcap"
            # print(file_path)
            csi = read_csi(file_path)
        # 80 MHz
            nullsubcarriers  = np.array([x+128 for x in [-128, -127, -126, -125, -124, -123, -1, 0, 1, 123, 124, 125, 126, 127]])
            pilotsubcarriers = np.array([x+128 for x in [-103, -75, -39, -11, 11, 39, 75, 103]])

        # 40 MHz
        #  nullsubcarriers = [x+64 for x in [-64, -63, -62, -61, -60, -59, -1, 0, 1, 59, 60, 61, 62, 63]]
        #  pilotsubcarriers = [x+64 for x in [11, 25, 53, -11, -25, -53]]

            csi=np.delete(csi,np.s_[nullsubcarriers],axis=1)
            csi=np.delete(csi,np.s_[pilotsubcarriers],axis=1)
            movement_data.append(csi)
        
   

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Shape of the input

In [26]:
x=np.array(movement_data)
x.shape

(720, 500, 234)

Creating train, validation and test inputs

In [27]:
x_train = []
x_valid = []
x_test = []
ind_list = []
for i in range(0, 720, 6):
    ind_list.append(i)
    # print(len(ind_list))
for i in range(0, 720):
    if(i in ind_list):
        x_test.append(x[i])
    # elif(i-1 in ind_list):
    #     x_valid.append(x[i])
        # print(i)
    else:
        x_train.append(x[i])



x_train = np.array(x_train)
x_valid = np.array(x_valid)
x_test = np.array(x_test)
print(len(x_train))
print(len(x_valid))
print(len(x_test))

600
0
120


Creating train, validation and test labels

In [28]:
y_train = []
for i in range(6):
    y_train.append([i for _ in range(80)])

y_train = np.array(y_train)
y_train.shape

(600,)

In [29]:
y_valid = []
for i in range(6):
    y_valid.append([i for _ in range(20)])

y_valid = np.array(y_valid)
y_valid.shape

(120,)

In [30]:
y_test = []
for i in range(6):
    y_test.append([i for _ in range(20)])

y_test = np.array(y_test)
y_test.shape

(120,)

Encoding Labels

In [31]:
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [32]:
y_train.shape

(600, 6)

In [33]:
y_valid.shape


(120, 6)

In [34]:
y_test.shape


(120, 6)

Input shape variables

In [35]:
input_length, input_features = 500, 234

Keras ConvLSTM Model

In [36]:
def create_model():
  model = Sequential()

  model.add(layers.Conv1D(filters=128, kernel_size=5, input_shape=(input_length, input_features)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(layers.Conv1D(filters=128, kernel_size=5, input_shape=(input_length, input_length)))
  model.add(BatchNormalization())
  model.add(Activation('tanh'))
  model.add(BatchNormalization())

  model.add(layers.MaxPool1D())

  model.add(layers.Bidirectional(layers.LSTM(200, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(200, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(200, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(200)))

  model.add(Dense(6))
  model.add(Activation('softmax'))

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model


In [37]:
# Clear clutter from previous session graphs.
tf.keras.backend.clear_session()

#train with TPU    
with strategy.scope():  
  model = create_model()
  history = model.fit(x_train, y_train, epochs=2000, batch_size=100, verbose=True)
  model.save('/content/gdrive/My Drive/Fition/colab1.h5')

Epoch 1/2000
6/6 [==============================] - 29s 449ms/step - loss: 1.6694 - accuracy: 0.3967
Epoch 2/2000
6/6 [==============================] - 3s 447ms/step - loss: 1.4642 - accuracy: 0.4483
Epoch 3/2000
6/6 [==============================] - 3s 447ms/step - loss: 1.3036 - accuracy: 0.4917
Epoch 4/2000
6/6 [==============================] - 3s 448ms/step - loss: 1.1699 - accuracy: 0.5883
Epoch 5/2000
6/6 [==============================] - 3s 449ms/step - loss: 1.0300 - accuracy: 0.6200
Epoch 6/2000
6/6 [==============================] - 3s 447ms/step - loss: 0.8967 - accuracy: 0.6833
Epoch 7/2000
6/6 [==============================] - 3s 449ms/step - loss: 0.8593 - accuracy: 0.6850
Epoch 8/2000
6/6 [==============================] - 3s 448ms/step - loss: 0.7349 - accuracy: 0.7183
Epoch 9/2000
6/6 [==============================] - 3s 449ms/step - loss: 0.6443 - accuracy: 0.7650
Epoch 10/2000
6/6 [==============================] - 3s 447ms/step - loss: 0.6375 - accuracy: 0.761

KeyboardInterrupt: ignored

Evaluating Model

In [ ]:
model.evaluate(x_test, y_test, batch_size=100)

In [ ]:
pyplot.plot(history.history['loss'])
# pyplot.plot(history.history['val_loss'])
# pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
# pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

Get predicitons on test data

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

In [ ]:
y_test = np.argmax(y_test, axis=None)


In [ ]:
print("Classification Report: \n", classification_report(y_test, y_pred_classes))

In [ ]:
cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["nm","sitdown", "standup","walking","falling","getintobed"])

In [ ]:
disp.plot(cmap = "Blues")
plt.show()